In [1]:
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.cluster import KMeans

import opt_utils as opt
from opt_utils import * 

DATA_RAW = '../input/optiver-realized-volatility-prediction'
MODEL = '../dart_model'
TRAIN = '../dart_model/enc_p13_train.pkl'
sns.set()

def rdf(train): 
    train['spe'] = ((train['target'] - train['pred']) / train['target']) ** 2
    return np.sqrt((train['spe'].sum()) / train.shape[0])

ModuleNotFoundError: No module named 'plotly'

In [ ]:
train = pd.read_pickle(TRAIN)
oof_preds = np.load(os.path.join(MODEL, 'oof_predictions.npy'))
train['pred'] = oof_preds
train['spe'] = ((train['target'] - train['pred']) / train['target']) ** 2
train['raw_error'] = train['pred'] - train['target']
train['rv'] = train['log_return_realized_volatility']
print(rdf(train))
train.head(3)

In [ ]:
top_err = train.sort_values('spe', ascending=False)

In [ ]:
top_err[:100].raw_error.hist()

In [ ]:
top_err[:100]['target'].mean()

In [ ]:
train.columns.tolist()

In [ ]:
train[['ratio', 'target']].corr()

In [ ]:
top_err[:100]

In [ ]:
top_err[:100][['ratio', 'target']].corr()

Lets see the distribution

In [ ]:
top_err[:100].sum_bid_ask_mean.mean()

In [ ]:
train.sum_bid_ask_mean.hist()

In [ ]:
train.corrwith(train['target']).abs().sort_values(ascending=False)[:20]

In [ ]:
sns.kdeplot(train['spe'])

What the heck. This looks like I have one or a few large errors and most near zero. 

In [ ]:
train['spe'].hist()

In [ ]:
print(train['spe'].max(), train['spe'].min())

In [ ]:
train['spe'].sort_values(ascending=False)

Lets look at the cumulative sum of errors to see how spread out the errors are. 

In [ ]:
df = train['spe'].sort_values().cumsum().reset_index(drop=True)
df.index /= df.index.values[-1]
df /= df.max()
df.plot()
plt.suptitle('Cumulative sum of sorted errors')
plt.xlabel('fraction of rows')
plt.ylabel('fraction of error')
plt.show()

It looks like the last 20% of the rows carry about 75% of the error. 
Lets see if the error is balanced accross stock_ids and time_ids. 

we can see whats going on with the bad erros. 

In [ ]:
print('Errors by stock_id sorted in descending order')
stock = train.groupby('stock_id')['spe'].sum()
stock.sort_values(ascending=False)

In [ ]:
train[train.stock_id == 31][['log_return_realized_volatility', 'target']].corr()

I was expecting the minimum to be much much lower compared to the maximum spe stock_id. The worst offender, stock_id 31 is only about 3 times worse than the second at 380 and gradually goes down to around 100 for the lowest stock. So stock 31 needs a little special attention, but we need to look at all stocks erros. I am guessing that the time_id aggregations won't be as balanced. 

In [ ]:
print('Errors by time_id sorted in descending order')
time = train.groupby('time_id')['spe'].sum()
worst_times = time.sort_values(ascending=False)
worst_times[:30]

In [ ]:
train[train.time_id == 25504][['log_return_realized_volatility', 'target']].corr()

In [ ]:
top_5_worst_times = worst_times[:5].index

There is a much greater imbalance here, with the top time_id accruing about 200 time the error of the bottom. If we can identify the reason for these errors in the hardest hit time ids, we have the best chance to improve our models. 

I am suspecting that either the first 10 minute window is low volatitliy, followed by a second 10 minute window with high volatility, or vica versa. For the top time_id, 25504, lets see how the realized volatility compares to the average of the first 10 minutes, and then how the target compares to the average target.

Lets normalize the realized the target and realized volatility of the the first 10 minutes. Then we can compare in the same scale that correlation is calculated in.

In [ ]:
tar_mean = train['target'].mean()
rv_mean = train.log_return_realized_volatility.mean()
tar_std = train['target'].std()
rv_std = train.log_return_realized_volatility.std()
train['nrv'] = (train.log_return_realized_volatility - rv_mean) / rv_std
train['ntar'] = (train.target - tar_mean) / tar_std

Lets look at the top 3 worst scoring time_ids, and then the best scoring time_ids.

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(15, 20))
train[train.time_id == 25504][['nrv', 'ntar']].plot(ax=axes[0])
axes[0].set_title('Worst scoring time_id, 25504 normalized target and realized variance')

train[train.time_id == 27174][['nrv', 'ntar']].plot(ax=axes[1])
axes[1].set_title('2nd worst scoring time_id, 27174 normalized target and realized variance')

train[train.time_id == 24034][['nrv', 'ntar']].plot(ax=axes[2])
axes[2].set_title('3rd worst scoring time_id, 24034 normalized target and realized variance')
plt.show()

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(15, 20))
train[train.time_id == 29850][['nrv', 'ntar']].plot(ax=axes[0])
axes[0].set_title('Best scoring time_id, 29850 normalized target and realized variance')

train[train.time_id == 4432][['nrv', 'ntar']].plot(ax=axes[1])
axes[1].set_title('2nd best scoring time_id, 4432 normalized target and realized variance')

train[train.time_id == 21116][['nrv', 'ntar']].plot(ax=axes[2])
axes[2].set_title('3rd best scoring time_id, 21116 normalized target and realized variance')

plt.show()

I am a bit confused. The 3rd worst scoring time_id looks really bad, but the first 2 look almost like best scoring time_ids. Maybe there is something I can't see yet that is huring my model. Lets plot the same plots again, along with our predictions. We will have to scale the predictions with the the same mean and std of the target to put them on the same scale. 

In [ ]:
train['norm_pred'] = (train.pred - tar_mean) / tar_std

fig, axes = plt.subplots(3, 1, figsize=(15, 20))
train[train.time_id == 25504][['nrv', 'ntar', 'norm_pred']].plot(ax=axes[0])
axes[0].set_title('Worst scoring time_id, 25504 normalized target and realized variance')

train[train.time_id == 27174][['nrv', 'ntar', 'norm_pred']].plot(ax=axes[1])
axes[1].set_title('2nd worst scoring time_id, 27174 normalized target and realized variance')

train[train.time_id == 24034][['nrv', 'ntar', 'norm_pred']].plot(ax=axes[2])
axes[2].set_title('3rd worst scoring time_id, 24034 normalized target and realized variance')
plt.show()

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(15, 20))
train[train.time_id == 29850][['nrv', 'ntar', 'norm_pred']].plot(ax=axes[0])
axes[0].set_title('Best scoring time_id, 29850 normalized target and realized variance')

train[train.time_id == 4432][['nrv', 'ntar', 'norm_pred']].plot(ax=axes[1])
axes[1].set_title('2nd best scoring time_id, 4432 normalized target and realized variance')

train[train.time_id == 21116][['nrv', 'ntar', 'norm_pred']].plot(ax=axes[2])
axes[2].set_title('3rd best scoring time_id, 21116 normalized target and realized variance')

plt.show()

I am still confused about the third worst time_id. It looks like the predictions follow the first 10 minute volatility, and should be far off. Maybe I am missing something by normalizing. Lets just look at the raw values of preds and the target, along with the error plotted underneath for reference. 


In [ ]:
fig, axes = plt.subplots(6, 1, figsize=(15, 35))
train[train.time_id == 25504][['target', 'pred']].plot(ax=axes[0])
axes[0].set_title('Worst scoring time_id, 25504 normalized target and realized variance')
train[train.time_id == 25504][['spe']].plot(ax=axes[1])

train[train.time_id == 27174][['target', 'pred']].plot(ax=axes[2])
axes[2].set_title('2nd worst scoring time_id, 27174 normalized target and realized variance')
train[train.time_id == 27174][['spe']].plot(ax=axes[3])

train[train.time_id == 24034][['target', 'pred']].plot(ax=axes[4])
axes[4].set_title('3rd worst scoring time_id, 24034 normalized target and realized variance')
train[train.time_id == 24034][['spe']].plot(ax=axes[5])

plt.show()

It is finally clear. The 3rd worst time_id has a consistent error accross all stocks. However, the target value is higher, so the squared percentage error does not blow up as it does in the first and second worst time_ids. Those time_ids only have one error each. The asolute value of the error is not very large, but since are dividing by the square of the true value, the fact that the target is small results in a huge error around 200 spe.

Lets look at top errors in general and see if they all share this quality of an extremely low target.

Lets start with the top 50 errors

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(15, 10))
top = train.sort_values('spe', ascending=False).head(50).reset_index()
top[['target', 'pred', 'log_return_realized_volatility']].plot(ax=axes[0])
axes[0].set_title('Top 25 squared percentage errors')
top[['spe']].plot(ax=axes[1])
axes[1].set_title('spe')
plt.show()

It definitely seems that the largest errors are coming from overpredicting the target, along with an especially small target. 
I think overprediction could be the only times when we have a large penalty from the metric. Lets look at the largest losses where we underpredict.

In [ ]:
df = train[train.pred < train.target]
df.sort_values('spe', ascending=False)[['pred', 'target', 'spe']].head(3)

The largest spe is less than .92. For reference, if we predicted 0 for all targets our average spe and overall metric would be 1. Given that the underpredicted errors gave spe penalties in the 5,6, and 7 range at the high end (highest about 200) with relatively small absolute error, and these underpredictions give a max penalty less than 1, I am certain that overpredicting especially small targets is the largest source of error. 

Lets look at some of the book and trade data for the top losses. Maybe we can find some features that show that we are about to enter a low volatility period, even if the previous 10 minutes was high.

I want to see from which part of the distributio coming from

In [ ]:
def add_percentile(df, col): 
    """"""
    if type(col) == list: 
        for c in col: df = add_percentile(df, c)
    else: 
        df[col + '_percentile'] = (df[col].rank(pct=True) * 100).astype(int)
    return df

In [ ]:
add_percentile(train, ['pred', 'target', 'log_return_realized_volatility'])
top_spe = train.sort_values('spe', ascending=False)
# [
#     ['stock_id', 'time_id', 'rv',
#      'target', 'pred', 'spe', 'log_return_realized_volatility_percentile', 'target_percentile', 'pred_percentile']
# ]
top_spe.head()

Adding the ratio of the volume of shares traded to the average total bid_size and ask_size because I think it could be important

In [ ]:
train['ratio'] = train.size_sum / train.sum_bid_ask_mean
train[['ratio']].corrwith(train['target'])

In [ ]:
df = load_bt(DATA_RAW, 31, 'train')

In [ ]:
df.head(1)

In [ ]:
df.groupby('time_id')['bid_price1', 'ask_price1'].transform('diff')

In [ ]:
df = load_bt(DATA_RAW, 31, 'train')
df.head()

In [ ]:
df['bid_price_diff'] = df.groupby('time_id')['bid_price1'].diff()
df.head()

In [ ]:
df.groupby('time_id')['bid_price_diff'].agg(count_unique)

In [ ]:
def plot_book(stock_id, time_id, train=train): 
    mask = (train.stock_id == stock_id) & (train.time_id == time_id)
    target, pred, spe, realized_volatitlity, pred_percentile, target_percentile, \
    realized_volatitlity_percentile, size_sum, ratio \
        = train.loc[mask, ['target', 'pred', 'spe', 
                           'log_return_realized_volatility',
                           'pred_percentile', 
                           'target_percentile', 
                           'log_return_realized_volatility_percentile', 
                           'size_sum', 
                           'ratio'
                          ]].values[0].round(5)
    
    bt = load_bt(DATA_RAW, stock_id, 'train')
    bt['wap'] = (bt['bid_price1'] * bt['ask_size1'] + bt['ask_price1'] * bt['bid_size1']) /\
                          (bt['bid_size1'] + bt['ask_size1'])
    bt['total_bid_size'] = bt['bid_size1'] + bt['bid_size2']
    bt['total_ask_size'] = bt['ask_size1'] + bt['ask_size2']
    book = bt[bt.time_id == time_id].set_index('seconds_in_bucket')
    
    fig, axes = plt.subplots(3, 1, figsize=(15, 15))
    
    cmap = {'ask_price1': 'yellow', 
            'ask_price2': 'khaki',
            'ask_size1': 'yellow', 
            'ask_size2': 'khaki',
            'bid_price1': 'blue', 
            'bid_price2': 'lightblue', 
            'bid_size1': 'blue', 
            'bid_size2': 'lightblue',
            'total_ask_size': 'yellow', 
            'total_bid_size': 'blue', 
            'wap': 'green', 
            'size': 'red', 
            'price': 'red', 
           }
    
    book[['ask_price2', 'ask_price1', 'bid_price1',  'bid_price2', 'wap', 'price']].plot(ax=axes[0], color=cmap)
    book[['bid_size1', 'ask_size1', 'bid_size2','ask_size2']].plot(ax=axes[1], color=cmap)
    book[['total_bid_size', 'total_ask_size']].plot(ax=axes[2], color=cmap)
    book[['size']].plot(marker='o', ax=axes[1], color=cmap)

    
    axes[0].set_title('Price info')
    axes[1].set_title('Size info')
    axes[2].set_title('Total size info')
    axes[0].legend(loc='upper left')
    axes[1].legend(loc='upper left')
    axes[2].legend(loc='upper left')
    
    fig.suptitle(f'\
        Stock: {stock_id}         missing seconds: {600 - len(book)}\n\
        Time_id: {time_id}\n\
        spe: {spe} \n\
        target: {target} \n\
        pred: {pred} \n\
        realized_volatitlity: {realized_volatitlity},\n\n\
        error: {round(pred - target, 5)} \n\
        target_percentile: {target_percentile} \n\
        pred_percentile: {pred_percentile} \n\
        realized_volatitlity_percentile: {realized_volatitlity_percentile}           total sales: {size_sum}         ratio: {ratio}\n')
    plt.tight_layout()
    plt.show()

In [ ]:
train['d'] = train['pred'] - train['target']

In [ ]:
for stock_id, time_id in train.sort_values('d', ascending=False)[['stock_id', 'time_id']].values[: 10]: 
    plot_book(stock_id, time_id)

In [ ]:
def mean_decay(x, decay=.9, step=-1, axis=0): 
    """Returns sum with exponential decay, step = -1
    for the end of the array to matter the most."""
    weights = np.power(decay, np.arange(x.shape[axis])[::step]).astype(np.float32)
    return np.sum(weights * x, axis=axis) / weights.sum()

In [ ]:
mean_decay(np.array([1,1,1,1]))

Let look at the top 3 instances for each of the ftop 5 lowest scoring time_ids

In [ ]:
for t in top_5_worst_times: 
    tmp = train[train.time_id == t].sort_values('spe', ascending=False)
    for stock_id, time_id in tmp[['stock_id', 'time_id']].values[: 3]: 
        plot_book(stock_id, time_id)
    

Lets look at the most penalized instances and see if we can see a pattern. 

In [ ]:
top = train.sort_values('spe', ascending=False)
display(train.sum_bid_ask_mean.mean())
display(top.iloc[:int(top.shape[0] * .2)].sum_bid_ask_mean.mean())
display(top.iloc[:int(top.shape[0] * .05)].sum_bid_ask_mean.mean())
display(top.iloc[:int(top.shape[0] * .01)].sum_bid_ask_mean.mean())

In [ ]:
top = train.sort_values('spe', ascending=False)
display(train.ratio.mean())
display(top.iloc[:int(top.shape[0] * .2)].ratio.mean())
display(top.iloc[:int(top.shape[0] * .05)].ratio.mean())
display(top.iloc[:int(top.shape[0] * .01)].ratio.mean())

In [ ]:
display(train[['sum_bid_ask_mean']].corrwith(train['spe']))
display(train[['sum_bid_ask_mean']].corrwith(train['target']))
display(train[['size_sum']].corrwith(train['spe']))
display(train[['size_sum']].corrwith(train['target']))
display(train[['ratio']].corrwith(train['spe']))
display(train[['ratio']].corrwith(train['target']))

In [ ]:
low = train.sort_values('log_return_realized_volatility')

Lets see what the lowest volatility instances look like before looking at the instances where we went from high volatility to a low target

In [ ]:
for stock_id, time_id in low[['stock_id', 'time_id']].values[: 10]: 
    plot_book(stock_id, time_id)

They all have a very low ratio of sales to ask and bid size. 

In [ ]:
for stock_id, time_id in top_spe[top_spe.stock_id != 31][['stock_id', 'time_id']].values[: 10]: 
    plot_book(stock_id, time_id)

In [ ]:
for stock_id, time_id in train[train.stock_id == 31][['stock_id', 'time_id']].values[: 10]: 
    plot_book(stock_id, time_id)

In [ ]:
comp = train[(train.log_return_realized_volatility_percentile < 60) & (train.log_return_realized_volatility_percentile > 40)]
for stock_id, time_id in comp[['stock_id', 'time_id']].values[:10]: 
    plot_book(stock_id, time_id)

In [ ]:
d = train.copy()
d.loc[d.ratio < 1, 'pred'] /= 2

In [ ]:
for stock_id, time_id in train[train.ratio < 1][['stock_id', 'time_id']].values[:12]: 
    plot_book(stock_id, time_id)

In [ ]:
rmspe(d['target'], d['pred'])

In [ ]:
train[train.ratio < 1]

In [ ]:
top_spe['vdiff'] = top_spe['log_return_realized_volatility'] - top_spe['target']
top_spe

In [ ]:
top_spe.sort_values('vdiff')

In [ ]:
df_train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')

In [ ]:
def visualize_book_time_bucket(stock_id, time_id):
    
    time_bucket = (df_train['stock_id'] == stock_id) & (df_train['time_id'] == time_id)
    
    target = df_train.loc[time_bucket, 'target'].iloc[0]
    realized_volatility = df_train.loc[time_bucket, 'realized_volatility_from_wap1'].iloc[0]
    df_book = read_book_data('train', stock_id, sort=True, forward_fill=True)
    df_book = df_book.set_index('seconds_in_bucket')
    
    df_book['wap1'] = (df_book['bid_price1'] * df_book['ask_size1'] + df_book['ask_price1'] * df_book['bid_size1']) /\
                      (df_book['bid_size1'] + df_book['ask_size1'])
    df_book['wap2'] = (df_book['bid_price2'] * df_book['ask_size2'] + df_book['ask_price2'] * df_book['bid_size2']) /\
                      (df_book['bid_size2'] + df_book['ask_size2'])
    
    fig, axes = plt.subplots(figsize=(32, 30), nrows=2)
    
    axes[0].plot(df_book.loc[df_book['time_id'] == time_id, 'bid_price1'], label='bid_price1', lw=2, color='tab:green')
    axes[0].plot(df_book.loc[df_book['time_id'] == time_id, 'ask_price1'], label='ask_price1', lw=2, color='tab:red')
    axes[0].plot(df_book.loc[df_book['time_id'] == time_id, 'bid_price2'], label='bid_price2', alpha=0.3, color='tab:green')
    axes[0].plot(df_book.loc[df_book['time_id'] == time_id, 'ask_price2'], label='ask_price2', alpha=0.3, color='tab:red')
    axes[0].plot(df_book.loc[df_book['time_id'] == time_id, 'wap1'], label='wap1', lw=2, linestyle='--', color='tab:blue')
    axes[0].plot(df_book.loc[df_book['time_id'] == time_id, 'wap2'], label='wap2', alpha=0.3, linestyle='--',  color='tab:blue')
    
    axes[1].plot(df_book.loc[df_book['time_id'] == time_id, 'bid_size1'], label='bid_size1', lw=2, color='tab:green')
    axes[1].plot(df_book.loc[df_book['time_id'] == time_id, 'ask_size1'], label='ask_size1', lw=2, color='tab:red')
    axes[1].plot(df_book.loc[df_book['time_id'] == time_id, 'bid_size2'], label='bid_size2', alpha=0.3, color='tab:green')
    axes[1].plot(df_book.loc[df_book['time_id'] == time_id, 'ask_size2'], label='ask_size2', alpha=0.3, color='tab:red')
    
    for i in range(2):
        axes[i].legend(prop={'size': 18})
        axes[i].tick_params(axis='x', labelsize=20, pad=10)
        axes[i].tick_params(axis='y', labelsize=20, pad=10)
    axes[0].set_ylabel('price', size=20, labelpad=15)
    axes[1].set_ylabel('size', size=20, labelpad=15)
    
    axes[0].set_title(
        f'Prices of stock_id {stock_id} time_id {time_id} - Current Realized Volatility: {realized_volatility:.6f} - Next 10 minute Realized Volatility: {target:.6f}',
        size=25,
        pad=15
    )
    axes[1].set_title(
        f'Sizes of stock_id {stock_id} time_id {time_id} - Current Realized Volatility: {realized_volatility:.6f} - Next 10 minute Realized Volatility: {target:.6f}',
        size=25,
        pad=15
    )
    
    plt.show()

In [ ]:
std = train['log_return_realized_volatility'].std()
# mean = train['log_return_realized_volatility'].mean()
# clip = mean + 3 * std
# clipped = train[train.]

# for x in [.99, .95, .9, .85, .80, .75.65]: 
#     tmp = train.copy()
#     tmp.loc[tmp.stock_id == 31, 'pred'] *= x
#     print(x, rmspe(tmp))

# for x in [.99, .95, .9, .85, .80, .75, .65]: 
#     tmp = train.copy()
#     tmp.loc[:, 'pred'] *= x
#     print(x, rmspe(tmp))

# # Lets see if we can make simple groupings to improve the baseline prediction of real volatility

# df = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
# df = df.merge(train[['stock_id', 'time_id', 'log_return_realized_volatility']], on=['stock_id', 'time_id'])
# df.columns = ['stock_id', 'time_id', 'target', 'rv']
# df['pred'] = df['rv']
# display(rmspe(df))

# df['stock_mean'] = df.groupby('stock_id')['rv'].transform('mean')
# df['time_mean'] = df.groupby('time_id')['rv'].transform('mean')
# df['pred'] = df['stock_mean']
# display(rmspe(df))

# m = df.rv.mean()
# df['time_ratio'] = df['time_mean'] / m
# df['pred'] = df['stock_mean'] * df['time_ratio']
# display(rmspe(df))

# df['rv_ratio'] = df['rv'] / df['stock_mean']
# df['pred'] = (df['rv_ratio'] + df['time_ratio']) * df['stock_mean'] / 2
# display(rmspe(df))

# None of the ideas could improve the baseline

# train.columns.tolist()

# c = np.power(.99, np.arange(10)[::-1]).sum()
# c = train['real_vol_mean_decay_0.85_-1'].mean() / m
# train['pred'] = train['real_vol_mean_decay_0.85_-1'] / (c * 1.2)
# rmspe(train)

# p = pd.read_pickle('../input/opt-feature-gen-p5-v2/p5_v2_train.pkl')

# c = np.power(.99, np.arange(10)[::-1]).sum()
# c = p['log_return_rv_99'].mean() / m 
# p['pred'] = p['log_return_rv_99'] / (c * 1.3) 
# rmspe(p)

# piv = df.pivot('time_id', 'stock_id', 'rv')
# print('columns in order: ', (all(piv.columns == sorted(piv.columns))))
# corr = piv.corr()

# for n in range(1, 15):
#     km = KMeans(n)
#     km.fit(corr)
#     mapper = dict(zip(corr.columns, km.labels_))
#     df['km_lab'] = df['stock_id'].map(mapper)
#     df['pred'] = df.groupby('km_lab')['rv'].transform('mean') * df['time_ratio']
#     print(n, rmspe(df))



# Verifying that the RMSPE is same as cv